In [1]:
from pathlib import Path
import polars as pl
from ebrec.utils._constants import (
    DEFAULT_HISTORY_ARTICLE_ID_COL, 
    DEFAULT_INVIEW_ARTICLES_COL,
    DEFAULT_IMPRESSION_ID_COL,
    DEFAULT_SUBTITLE_COL,
    DEFAULT_LABELS_COL,
    DEFAULT_TITLE_COL, 
    DEFAULT_USER_COL, 
)
#
from ebrec.utils._behaviors import (
    create_binary_labels_column, 
    sampling_strategy_wu2019,
    add_known_user_column,
    add_prediction_scores,
    truncate_history, 
)
from ebrec.utils._articles import convert_text2encoding_with_transformers
from ebrec.utils._articles import create_article_id_to_value_mapping
from ebrec.utils._nlp import get_transformers_word_embeddings
from ebrec.utils._polars import concat_str_columns
#
from ebrec.models.newsrec.dataloader import NRMSDataLoader
from ebrec.models.newsrec.model_config import hparams_nrms
from ebrec.models.newsrec import NRMSModel
#
from transformers import AutoTokenizer, AutoModel
import tensorflow as tf

# Getting started

## Load dataset

In [2]:
# 
def ebnerd_from_path(path:Path, history_size:int = 30) -> pl.DataFrame:
    """
    Load ebnerd - function 
    """
    df_history = (
        pl.scan_parquet(path.joinpath("history.parquet"))
        .select(DEFAULT_USER_COL, DEFAULT_HISTORY_ARTICLE_ID_COL)
        .pipe(
            truncate_history,
            column=DEFAULT_HISTORY_ARTICLE_ID_COL,
            history_size=history_size,
            padding_value=0,
        )
    )
    df_behaviors = (
        pl.scan_parquet(path.joinpath("behaviors.parquet"))
        .join(df_history, on=DEFAULT_USER_COL, how="inner")
        .collect()
    )
    return df_behaviors

## Generate labels
We sample a few just to get started. For testset we just make up a dummy column with 0 and 1 - this is not the true labels.

In [23]:
path = Path("../downloads/demo")
HISTORY_SIZE = 30
N_SAMPLES = 100
df_train = (
    ebnerd_from_path(path.joinpath("train"), history_size=HISTORY_SIZE)
    .pipe(sampling_strategy_wu2019,npratio=4,shuffle=True,with_replacement=True,seed=123,)
    .pipe(create_binary_labels_column)
    .sample(n=N_SAMPLES)
)
# =>
df_validation = (
    ebnerd_from_path(path.joinpath("validation"), history_size=HISTORY_SIZE)
    .pipe(create_binary_labels_column)
    .sample(n=N_SAMPLES)
)
# =>
df_test = (
    ebnerd_from_path(path.joinpath("test"), history_size=HISTORY_SIZE)
    .with_columns(
        pl.col(DEFAULT_INVIEW_ARTICLES_COL)
        .list.eval(pl.element() * 0)
        .list.tail(-1)
        .list.eval(pl.element().extend_constant(1, n=1))
        .alias(DEFAULT_LABELS_COL)
    )
    .sample(n=N_SAMPLES)
)

/Users/johannes.kruse/Documents/CodeProject/ebnerd-benchmark/src/ebrec/utils/_behaviors.py:621: UserWarning: truncate_history: The history IDs expeced in ascending order
  warnings.warn(f"{function_name}: The history IDs expeced in ascending order")


### Training/Validation annd Testset
Note, the testset doesn't include labels, and we have remove some of the other columns.

In [24]:
df_train.head(2)

impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage,article_id_fixed,labels
u32,i32,datetime[μs],f32,f32,i8,list[i64],list[i64],u32,bool,i8,i8,i8,bool,u32,f32,f32,list[i32],list[i8]
360352480,null,2023-02-25 19:36:02,35.0,null,2,"[9650722, 9650487, … 9488931]",[9650722],483705,true,null,null,null,false,25649,36.0,null,"[9634978, 9550338, … 9634995]","[1, 0, … 0]"
475719166,null,2023-02-28 05:06:56,12.0,null,2,"[9653455, 9653474, … 9653474]",[9653238],2036647,false,null,null,null,false,94813,31.0,61.0,"[9631127, 9635113, … 9646969]","[0, 0, … 0]"


In [25]:
df_test.head(2)

impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,article_id_fixed,labels
u32,i32,datetime[μs],f32,f32,i8,list[i32],u32,bool,i8,i8,i8,bool,u32,list[i32],list[i32]
606117094,null,2023-03-09 14:59:46,35.0,null,1,"[9668418, 9640339, … 9658841]",1590380,false,null,null,null,false,18378,"[9667845, 9665316, … 9667458]","[0, 0, … 1]"
121834159,null,2023-03-11 15:46:20,38.0,null,2,"[9670853, 9671498, … 8996121]",597150,false,null,null,null,false,121652,"[9664629, 9663520, … 9667835]","[0, 0, … 1]"


## Load articles

In [26]:
df_articles = pl.read_parquet(path.joinpath("articles.parquet"))
df_articles.head(2)

article_id,title,subtitle,last_modified_time,premium,body,published_time,image_ids,article_type,url,ner_clusters,entity_groups,topics,category,subcategory,category_str
i32,str,str,datetime[μs],bool,str,datetime[μs],list[i64],str,str,list[str],list[str],list[str],i16,list[i16],str
3040067,"""Det kostede en…","""Annette Strøyb…",2023-06-29 06:21:06,false,"""- Jeg fik Nath…",2005-01-28 12:50:00,"[3180893, 3069163]","""article_defaul…","""https://ekstra…",[],[],[],414,[432],"""underholdning"""
3043185,"""Venter firling…","""Dale Chalk vil…",2023-06-29 06:21:14,false,"""Kronprinsesse …",2005-06-02 05:30:00,"[3067843, 3046322]","""article_defaul…","""https://ekstra…",[],[],[],118,[133],"""nyheder"""


### Init model using HuggingFace's tokenizer and wordembedding
In the original implementation, they use the GloVe embeddings and tokenizer. To get going fast, we'll use a multilingual LLM from Hugging Face. 
Utilizing the tokenizer to tokenize the articles and the word-embedding to init NRMS.


In [27]:
TRANSFORMER_MODEL_NAME = "bert-base-multilingual-cased"
TEXT_COLUMNS_TO_USE = [DEFAULT_SUBTITLE_COL, DEFAULT_TITLE_COL]
MAX_TITLE_LENGTH = 30

# LOAD HUGGINGFACE:
transformer_model = AutoModel.from_pretrained(TRANSFORMER_MODEL_NAME)
transformer_tokenizer = AutoTokenizer.from_pretrained(TRANSFORMER_MODEL_NAME)

# We'll init the word embeddings using the 
word2vec_embedding = get_transformers_word_embeddings(transformer_model)
# 
df_articles, cat_cal = concat_str_columns(df_articles, columns=TEXT_COLUMNS_TO_USE)
df_articles, token_col_title = convert_text2encoding_with_transformers(
    df_articles, transformer_tokenizer, cat_cal, max_length=MAX_TITLE_LENGTH
)
# =>
article_mapping = create_article_id_to_value_mapping(df=df_articles, value_col=token_col_title)

# Use the dataloader setup.
We have disconnected the models and data. Hence, built a dataloader that fits your purpose.

In [28]:
train_dataloader = NRMSDataLoader(
    behaviors=df_train,
    article_dict=article_mapping,
    unknown_representation="zeros",
    history_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
    eval_mode=False,
    batch_size=64,
)
val_dataloader = NRMSDataLoader(
    behaviors=df_validation,
    article_dict=article_mapping,
    unknown_representation="zeros",
    history_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
    eval_mode=True,
    batch_size=32,
)
test_dataloader = NRMSDataLoader(
    behaviors=df_test,
    article_dict=article_mapping,
    unknown_representation="zeros",
    history_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
    eval_mode=True,
    batch_size=32,
)

## Train the model


In [29]:
MODEL_NAME = "NRMS"
LOG_DIR = f"downloads/runs/{MODEL_NAME}"
MODEL_WEIGHTS = f"downloads/data/state_dict/{MODEL_NAME}/weights"

# CALLBACKS
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR, histogram_freq=1)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=2)
modelcheckpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=MODEL_WEIGHTS, save_best_only=True, save_weights_only=True, verbose=1
)

hparams_nrms.history_size = HISTORY_SIZE
model = NRMSModel(
    hparams=hparams_nrms,
    word2vec_embedding=word2vec_embedding,
    seed=42,
)
hist = model.model.fit(
    train_dataloader,
    validation_data=val_dataloader,
    epochs=1,
    callbacks=[tensorboard_callback, early_stopping, modelcheckpoint],
)
model.model.load_weights(filepath=MODEL_WEIGHTS)

2/2 [==============================] - ETA: 0s - loss: 1.6088
Epoch 1: val_loss improved from inf to 0.00000, saving model to downloads/data/state_dict/NRMS/weights
2/2 [==============================] - 64s 62s/step - loss: 1.6088 - val_loss: 0.0000e+00


# Predict on the testset

In [30]:
pred = model.scorer.predict(test_dataloader)

4/4 [==============================] - 6s 1s/step


In [32]:
df_test = add_prediction_scores(df_test, pred.tolist()).pipe(
    add_known_user_column, known_users=df_train[DEFAULT_USER_COL]
)

# Example how to compute some metrics:

In [35]:
from ebrec.evaluation import AucScore, MrrScore, MetricEvaluator, NdcgScore
def compute_evaluation_scores(
    df: pl.DataFrame,
    metric_functions: list[MetricEvaluator] = [
        AucScore(),
        MrrScore(),
        NdcgScore(k=5),
        NdcgScore(k=10),
    ],
    pred_score = "scores",
    labels:str="labels"
) -> dict[str, float]:
    # =>
    y_pred = df[pred_score].to_list()
    y_true = df[labels].to_list()
    # =>
    metr = MetricEvaluator(
        labels=y_true,
        predictions=y_pred,
        metric_functions=metric_functions,
    )
    return metr.evaluate().evaluations

compute_evaluation_scores(df_test)


{'auc': 0.4516887234900142,
 'mrr': 0.3148457418185679,
 'ndcg@5': 0.35970396106087243,
 'ndcg@10': 0.41935821814111035}